In [3]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
from langgraph.checkpoint.memory import InMemorySaver

In [4]:
load_dotenv()

True

In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

In [7]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [8]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [9]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [10]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [11]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!',
 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting kneaded!" is a pun based on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working and stretching dough to develop gluten, which gives pizza its texture. A pizza maker literally kneads the dough.\n\n*   **Figurative meaning:** "Kneaded" can also be used figuratively to mean "stressed" or "harassed." Someone who is constantly being asked to do things, or who is under a lot of pressure, might say they feel "kneaded" or "pushed around."\n\nThe humor comes from the pizza maker being "tired of getting kneaded" in both senses of the word. He\'s physically tired of the labor of kneading dough, but also figuratively tired of the pressure and demands of the job. The setup leads you to expect a serio

In [12]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting kneaded!" is a pun based on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working and stretching dough to develop gluten, which gives pizza its texture. A pizza maker literally kneads the dough.\n\n*   **Figurative meaning:** "Kneaded" can also be used figuratively to mean "stressed" or "harassed." Someone who is constantly being asked to do things, or who is under a lot of pressure, might say they feel "kneaded" or "pushed around."\n\nThe humor comes from the pizza maker being "tired of getting kneaded" in both senses of the word. He\'s physically tired of the labor of kneading dough, but also figuratively tired of the pressure and demands of the job. The setup leads yo

In [13]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting kneaded!" is a pun based on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working and stretching dough to develop gluten, which gives pizza its texture. A pizza maker literally kneads the dough.\n\n*   **Figurative meaning:** "Kneaded" can also be used figuratively to mean "stressed" or "harassed." Someone who is constantly being asked to do things, or who is under a lot of pressure, might say they feel "kneaded" or "pushed around."\n\nThe humor comes from the pizza maker being "tired of getting kneaded" in both senses of the word. He\'s physically tired of the labor of kneading dough, but also figuratively tired of the pressure and demands of the job. The setup leads y

In [15]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting kneaded!" is a pun based on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working and stretching dough to develop gluten, which gives pizza its texture. A pizza maker literally kneads the dough.\n\n*   **Figurative meaning:** "Kneaded" can also be used figuratively to mean "stressed" or "harassed." Someone who is constantly being asked to do things, or who is under a lot of pressure, might say they feel "kneaded" or "pushed around."\n\nThe humor comes from the pizza maker being "tired of getting kneaded" in both senses of the word. He\'s physically tired of the labor of kneading dough, but also figuratively tired of the pressure and demands of the job. The setup leads yo

In [18]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting kneaded!" is a pun based on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working and stretching dough to develop gluten, which gives pizza its texture. A pizza maker literally kneads the dough.\n\n*   **Figurative meaning:** "Kneaded" can also be used figuratively to mean "stressed" or "harassed." Someone who is constantly being asked to do things, or who is under a lot of pressure, might say they feel "kneaded" or "pushed around."\n\nThe humor comes from the pizza maker being "tired of getting kneaded" in both senses of the word. He\'s physically tired of the labor of kneading dough, but also figuratively tired of the pressure and demands of the job. The setup leads y

## Time Travel

In [19]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [21]:
# workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

In [22]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting kneaded!" is a pun based on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to the process of working and stretching dough to develop gluten, which gives pizza its texture. A pizza maker literally kneads the dough.\n\n*   **Figurative meaning:** "Kneaded" can also be used figuratively to mean "stressed" or "harassed." Someone who is constantly being asked to do things, or who is under a lot of pressure, might say they feel "kneaded" or "pushed around."\n\nThe humor comes from the pizza maker being "tired of getting kneaded" in both senses of the word. He\'s physically tired of the labor of kneading dough, but also figuratively tired of the pressure and demands of the job. The setup leads y

## Updating State

In [23]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f088718-ff7e-6660-8000-74b2a9fde065'}}

In [24]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f088718-ff7e-6660-8000-74b2a9fde065'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-09-03T02:56:17.655561+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='552c9fbf-49ff-aa01-cad3-5b74c5a41cd6', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting kneaded!" is a pun based on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to

In [26]:
# workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc72-ca16-6359-8001-7eea05e07dd2"}})

In [27]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f088718-ff7e-6660-8000-74b2a9fde065'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-09-03T02:56:17.655561+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='552c9fbf-49ff-aa01-cad3-5b74c5a41cd6', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting kneaded!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting kneaded!" is a pun based on the double meaning of the word "kneaded."\n\n*   **Literal meaning:** In the context of pizza making, "kneading" refers to

## Fault Tolerance

In [28]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [29]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [30]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [31]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))